In [16]:
import tensorflow as tf

## constant
常量

In [17]:
# a computational graph
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [11]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [15]:
node3 = tf.add(node1, node2)
print('node3: ', node3)
print('sess.run(node3): ',sess.run(node3))

node3:  Tensor("Add_12:0", shape=(), dtype=float32)
sess.run(node3):  7.0


## placeholder
类似print占位符

In [18]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [20]:
print(sess.run(adder_node, {a: 3, b: 4.5}))  # 类似赋值后执行
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]})) #list

7.5
[ 3.  7.]


In [21]:
add_and_triple = adder_node * 3.  # 乘一个常量3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


## Variable
变量，即可初始化，可改变（可训练参数）

In [55]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
print(linear_model)

Tensor("add_7:0", dtype=float32)


为了初始化所有变量，必须显式调用下列函数

In [56]:
init = tf.global_variables_initializer()
sess.run(init)

In [57]:
print('predict y: ', sess.run(linear_model, {x: [1,2,3,4]}))

predict y:  [ 0.          0.30000001  0.60000002  0.90000004]


In [58]:
# calculate loss value
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)  # square that error
loss = tf.reduce_sum(squared_deltas)  # sum all the squared errors
print('loss: ', sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

loss:  23.66


In [59]:
# tf.Variable can be changed using operations like tf.assign.
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, b])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

6.76


## tf.train API
### gradient descent

In [63]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
sess.run(init) # reset values to incorrect defaults.
for i in range(1000): # iter count = 1000
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## tf.estimator
tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:
- running traing loop
- running evaluation loops
- managing data sets

In [79]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

AttributeError: module 'tensorflow.python.estimator.estimator_lib' has no attribute 'LinearRegressor'

## A custom model
Provide Estimator a function model_fn that tells tf.estimator how it can evaluate predictions, training steps, and loss. 

In [80]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_model_dir': '/tmp/tmpuafsr4ga', '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpuafsr4ga/model.ckpt.
INFO:tensorflow:step = 1, loss = 10.8243056552
INFO:tensorflow:global_step/sec: 1311.17
INFO:tensorflow:step = 101, loss = 0.0218054139805 (0.078 sec)
INFO:tensorflow:global_step/sec: 1279.22
INFO:tensorflow:step = 201, loss = 0.000614853254618 (0.079 sec)
INFO:tensorflow:global_step/sec: 1243.02
INFO:tensorflow:step = 301, loss = 0.000115225258739 (0.080 sec)
INFO:tensorflow:global_step/sec: 1359.36
INFO:tensorflow:step = 401, loss = 4.54759817218e-05 (0.072 sec)
INFO:tensorflow:global_step/sec: 1287.86
INFO:tensorflow:step = 501, loss = 6.00554633202e-07 (0.078 s